# Predicting Air Quality Using Particulate

In [15]:
! ls data/

grid_metadata.csv
pm25_satellite_metadata.csv
train_labels.csv


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_regression
import sklearn.metrics as metrics
from scipy import stats as stats

from sklearn.metrics import r2_score, mean_absolute_error
import folium

In [8]:
grid = pd.read_csv('data/grid_metadata.csv')
satellite = pd.read_csv('data/pm25_satellite_metadata.csv')

In [7]:
grid.head()

,grid_id,location,tz,wkt
0,1X116,Taipei,Asia/Taipei,"POLYGON ((121.5257644471362 24.97766123020391,..."
1,1Z2W7,Delhi,Asia/Calcutta,"POLYGON ((77.30453178416276 28.54664454217707,..."
2,3S31A,Los Angeles (SoCAB),Etc/GMT+8,POLYGON ((-117.9338248256995 33.79558357488509...
3,6EIL6,Delhi,Asia/Calcutta,"POLYGON ((77.07995296313287 28.54664454217707,..."
4,7334C,Delhi,Asia/Calcutta,"POLYGON ((77.12486872733885 28.54664454217707,..."


In [13]:
grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   grid_id   54 non-null     object
 1   location  54 non-null     object
 2   tz        54 non-null     object
 3   wkt       54 non-null     object
dtypes: object(4)
memory usage: 1.8+ KB


In [14]:
grid.describe()

,grid_id,location,tz,wkt
count,54,54,54,54
unique,54,3,3,54
top,GJLB2,Delhi,Asia/Calcutta,"POLYGON ((121.6155959755481 25.01836939334328,..."
freq,1,33,33,1


In [9]:
satellite.head()

,granule_id,time_start,time_end,product,location,split,us_url,eu_url,as_url,cksum,granule_size
0,20180201T191000_maiac_la_0.hdf,2018-02-01T17:25:00.000Z,2018-02-01 19:10:00+00:00,maiac,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,911405771,10446736
1,20180202T195000_maiac_la_0.hdf,2018-02-02T18:05:00.000Z,2018-02-02 19:50:00+00:00,maiac,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,2244451908,11090180
2,20180203T203000_maiac_la_0.hdf,2018-02-03T17:10:00.000Z,2018-02-03 20:30:00+00:00,maiac,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,3799527997,12468482
3,20180204T194000_maiac_la_0.hdf,2018-02-04T17:55:00.000Z,2018-02-04 19:40:00+00:00,maiac,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,4105997844,13064424
4,20180205T202000_maiac_la_0.hdf,2018-02-05T17:00:00.000Z,2018-02-05 20:20:00+00:00,maiac,la,train,s3://drivendata-competition-airathon-public-us...,s3://drivendata-competition-airathon-public-eu...,s3://drivendata-competition-airathon-public-as...,1805072340,12549313


In [10]:
satellite.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7721 entries, 0 to 7720
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   granule_id    7721 non-null   object
 1   time_start    7721 non-null   object
 2   time_end      7721 non-null   object
 3   product       7721 non-null   object
 4   location      7721 non-null   object
 5   split         7721 non-null   object
 6   us_url        7721 non-null   object
 7   eu_url        7721 non-null   object
 8   as_url        7721 non-null   object
 9   cksum         7721 non-null   int64 
 10  granule_size  7721 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 663.6+ KB


In [12]:
satellite.describe()

,cksum,granule_size
count,7.721000e+03,7.721000e+03
mean,2.156026e+09,1.259338e+07
std,1.236418e+09,1.020110e+07
min,3.161110e+05,1.105365e+06
25%,1.092908e+09,6.093818e+06
50%,2.148802e+09,9.972825e+06
75%,3.233612e+09,1.309916e+07
max,4.294952e+09,5.140955e+07
